# Entrega 2 — Referencial, pipeline e cronograma

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fpaterni/projeto-aplicado-iv-desemprego-br/blob/main/notebooks/entrega2/cd_projeto_aplicado_IV_entrega_2.ipynb)


In [ ]:
# Instalação (Colab)
!pip -q install numpy pandas statsmodels scikit-learn matplotlib pmdarima
